In [ ]:
#安装MMDEPLOY ，这里只编译torchscript的自定义操作，后端操作要cmake处理
!git clone https://github.com/open-mmlab/mmdeploy.git
%cd mmdeploy
!export MMDEPLOY_DIR=/kaggle/working/mmdeploy
!pip install -e .

In [ ]:
#cmake编译自定义的OP
%cd /kaggle/working/mmdeploy/
!mkdir -p build
%cd build
!cmake -DCMAKE_CXX_COMPILER=g++ 
    -DMMDEPLOY_TARGET_BACKENDS=trt 
    -DTENSORRT_DIR=${TENSORRT_DIR} #这个必须设置
    -DCUDNN_DIR=${CUDNN_DIR} #这个可以没有
    ..
!make -j$(nproc)
!make install

# 安装库

In [ ]:
%cd /kaggle/working/
!git clone https://github.com/open-mmlab/mmclassification.git
%cd mmclassification
!pip install -e . 
#保存路径
!mkdir checkpoints
#下载checkpoint
!wget -c https://download.openmmlab.com/mmclassification/v0/resnet/resnet18_8xb32_in1k_20210831-fbbb1da6.pth \
      -O checkpoints/tmp.pth

In [ ]:
!python ./tools/deploy.py \
    /root/mmdeploy/configs/mmseg/segmentation_tensorrt_static-512x512.py \
    /root/mmsegmentation/configs/encnet/encnet_r50-d8_4xb4-80k_ade20k-512x512.py \
    /root/checkpoint/encnet_r50-d8_512x512_80k_ade20k_20200622_042412-44b46b04.pth \
   /root/mmsegmentation/resources/3dogs.jpg \
    --show \
    --device 'cuda:0'

# 执行

In [ ]:
import tensorrt as trt
assert trt.Builder(trt.Logger())
from PIL import Image
import numpy as np

In [ ]:
# load all custom plugins
trt.init_libnvinfer_plugins(logger,'')

with open('end2end.engine','rb') as f:
    engineString = f.read()
    
engine = trt.Runtime(logger).deserialize_cuda_engine(engineString) 

In [ ]:
context        =engine.create_execution_context()
context.set_binding_shape(0,[1,3,224,224])

In [ ]:
def test_vis(img_path):
    img=Image.open(img_path)
    img_norm_cfg = dict(mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
    resize = transforms.Resize([512,512])
    norm = transforms.Normalize(mean=img_norm_cfg['mean'],std=img_norm_cfg['std'])
    img = resize(img)
    to_tensor = transforms.ToTensor()
    img_y = to_tensor(img)
    #img_y = norm(img_y)
    img_y.unsqueeze_(0)
    img_y=img_y.cuda()
    out =torch.rand(1,1,512,512).cuda()
    context.execute_async_v2([img_y.data_ptr(),out.data_ptr()],0)
    label = out.cpu()
    label.squeeze_(0)
    label = label.permute(1,2,0)
    plt.figure(figsize=(20,20))
    plt.subplot(1,2,1)
    plt.imshow(img)
    plt.subplot(1,2,2)
    plt.imshow(label)
    plt.show()